### 1. 순환 신경망(Recurrent Neural Network, RNN)

* 은닉층의 노드에서 활성화함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고 있음

* RNN에서는 은닉층에서 활성화함수를 통해 결과를 내보내는 역할을 하는 모드를 셀(cell)이라고 함
* 이 셀은 이전의 값을 기억하는 일종의 메모리 역할을 수행하여 메모리 셀 또는 RNN 셀이라고 함

* 은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있음
* 현재 시점 t에서의 메모리 셀이 갖고 있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미함
* 이 메모리 셀이 출력층 방향 또는 다음 시점인 t+1의 자신에게 보내는 값을 은닉상태(hidden state)라고 함
* 즉, t 시점의 메모리 셀은 t-1 시점의 메모리 셀이 보낸 은닉상태값을 t 시점의 은닉상태 계산을 위해 입력값으로 사용
-----

* RNN은 입력과 출력의 길이를 다르게 설계할 수 있어 다양한 용도로 사용 가능
* 하나의 입력에 대해 여러개의 출력을 의미하는 일대다(one-to-many) 구조모델은 하나의 이미지 입력에 대해 사진의 제목을 출력하는 이미지 캡셔닝 작업에 사용 가능
* 다대일(many-to-one) 구조의 모델은 입력문서가 긍정적인지 부정적인지 판별하는 감성분류(sentiment classification) 또는 스팸메일 구분 등에 사용 가능
* 다대다 (many-to-many) 구조의 모델의 경우, 사용자가 문자을 입력하면 대답 문장을 출력하는 챗봇, 번역기, 태깅작업 등에 사용


----
* 은닉층에서 주로 사용하는 활성화함수는 하이퍼볼릭탄젠트 함수(tahn)가 사용됨


### 2. Keras로 RNN 구현

* hidden_units : 은닉상태의 크기 정의. 메모리 셀이 다음 시점의 메모리셀과 출력층으로 보내는 값의 크기(output_dim)과도 동일. 중소형 모델의 경우 128, 256, 512, 1024 등의 값을 가짐

* timesteps = 입력 시퀀스의 길이라고 표현. 시점의 수
* input_dim = 입력의 크기

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2, 10)))  
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


* 출력값이 (batch_size, output_dim)의 2D 텐서일 경우, output_dim은 hidden_units 값인 3임
* 이 경우, batch_size를 현단계에서 알수 없으므로 (None, 3)이 되며, 다음 예제는 batch_size를 정해본다.

In [3]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


* batch_size를 8로 기재하면 출력의 크기가 (8, 3)이 됨. return_sequences 매개 변수에 True를 기재하여 출력값으로 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴하도록 모델을 만들어 봄.

In [4]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10), return_sequences=True))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


* 모든 은닉층 값 리턴

### 3. 파이썬으로 RNN 구현하기

* 은닉상태를 계산하는 식부터 
$$h_{t} = tanh(W_{x}X_{t} + W_{h}h_{t−1} + b)$$

##### 가상코드

In [6]:
timesteps = 10
input_dim = 4
hidden_units = 8

# 입력에 해당하는 2D 텐서
inputs = np.random.random((timesteps, input_dim))
inputs.shape

(10, 4)

In [8]:
## 초기 은닉상태 초기화 :  은닉상태의 크기를 8로 설정
hidden_state_t = np.zeros((hidden_units,))
hidden_state_t

array([0., 0., 0., 0., 0., 0., 0., 0.])

* 가중치와 편향을 각 크기에 맞게 정의하고 크기를 출력

In [10]:
Wx = np.random.random((hidden_units, input_dim))  # (8, 4) 크기의 2D 텐서. 입력에 대한 가중치
Wh = np.random.random((hidden_units, hidden_units)) # (8, 8) 크기의 2D 텐서. 은낙상태 대한 가중치
b = np.random.random((hidden_units,)) # (8, )크기의 1D 텐서. 편향 값

print('가중치 Wx의 크기(shape) :',np.shape(Wx))
print('가중치 Wh의 크기(shape) :',np.shape(Wh))
print('편향의 크기(shape) :',np.shape(b))

가중치 Wx의 크기(shape) : (8, 4)
가중치 Wh의 크기(shape) : (8, 8)
편향의 크기(shape) : (8,)


In [15]:
total_hidden_states = []

# 각 시점별 입력값
for input_t in inputs:  ## inputs (10, 4)에 대해 10개 시점 구분
    output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)
    total_hidden_states.append(list(output_t))  # list 형태로 저장
    hidden_state_t = output_t
    
total_hidden_states[0], len(total_hidden_states)

([0.9999831451045942,
  0.9998963184915816,
  0.9999914196010343,
  0.9992991178246589,
  0.9999856683036175,
  0.9999663510622909,
  0.9999274860149827,
  0.9998078974427353],
 10)

In [16]:
# (timesteps, output_dim) ('모든 은닉시점의 은닉상태')
total_hidden_states = np.stack(total_hidden_states, axis=0)
total_hidden_states.shape

(10, 8)

In [17]:
total_hidden_states[0]

array([0.99998315, 0.99989632, 0.99999142, 0.99929912, 0.99998567,
       0.99996635, 0.99992749, 0.9998079 ])

### 4. 깊은 순환 신경망(Deep Recurrent Neural Network)

* 은닉층을 2개 이상 추가하는 경우

In [18]:
model = Sequential()
model.add(SimpleRNN(hidden_units, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_units, return_sequences=True))

In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 10, 8)             112       
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 10, 8)             136       
                                                                 
Total params: 248
Trainable params: 248
Non-trainable params: 0
_________________________________________________________________


### 5. 양방향 순환 신경망(Bidirectional Recurrent Neurual Network)

* 양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 입력뿐만 아니라, 이후 시점의 입력 또한 예측에 기여할 수 있다는 아이디어에 기반

* RNN이 풀고자 하는 문제 중에서는 과거 시점의 입력 뿐만 아니라 미래 시점의 입력에 힌트가 있는 경우도 많음. 그래서 이전과 이후의 시점 모두를 고려해서 현재 시점의 예측을 더욱 정확하게 할 수 있도록 고안된 것이 양방향 RNN임

---
* 따라서 하나의 출력값을 예측할 때 기본적으로 두개의 메모리 셀을 사용
* 첫번째 메모리셀은 앞서 나온 이전 시점의 은닉상태(Forward States)를 전달받아 현재의 은닉상태를 계산
* 두번째 메모리셀은 이후의 시점의 은닉상태(Backward States)를 전달받아 현재의 은닉상태를 계산. 입력시퀀스를 반대방향으로 읽어냄
* 이 두개의 값 모두가 현재 시점의 출력층에서 출력값을 예측하기 위해 사용됨

In [20]:
from tensorflow.keras.layers import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))

In [21]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 10, 16)           224       
 l)                                                              
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


* 은닉층을 추가한다고 해서 모델 성능이 좋아지는 것이 아님. 은닉층을 추가하면 학습량이 많아지지만, 그만큼 데이터도 필요

In [23]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))

In [24]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 10, 16)           224       
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
Total params: 1,424
Trainable params: 1,424
Non-traina